In [ ]:
pip install openai

In [ ]:
pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()
openai_key = os.environ.get('OPENAI_API_KEY')

client = OpenAI(
    api_key = openai_key
)

In [62]:
from dataclasses import dataclass

@dataclass
class ExternalKnowledge:
    title: str
    content: str
    
    def __str__(self) -> str:
        return f"{self.title}: {self.content}"

wafa = """
    Muhammad Wafa Al Ausath, or commonly known as Wafa, is a student majoring in Informatics Engineering at Lampung University.
    Wafa has a strong interest in the field of information technology, especially mobile application development.
    He has experience as a mobile application developer at Coolva Technology and has worked as a freelance mobile developer.
    He has also served as a mobile developer intern at several startup companies.
    
    github profile = https://github.com/mikirinkode
    linkedin profile = www.linkedin.com/in/muhammadwafaalausath
    personal website = https://mikirinkode.com
""".strip()

extra_knowledges = [
    ExternalKnowledge(title="Wafa",content=wafa),
]

In [68]:
class MyAssistant:
    def __init__(self, openai_client, extra_knowledges, name = "Miriko"):
        self.name = name
        self.messages = [
            {"role": "system", "content": f"""
             You are a Wafa's personal AI Assistant that helps him with his daily tasks. 
             Here some extra knowledges for you {extra_knowledges}
             """.strip()},
        ]
        self.client = openai_client
        print(self.greet())

    def greet(self):
        # print(self.chat("Say Greeting"))
        return f"Hello, {self.name} here. Anything i could help today?"

    # promt -> input user
    # system_content -> role the ai that we want
    # model -> used gpt model
    # stream -> to display the result in real time (like typing)
    def chat(self, prompt, system_content = None, model = "gpt-3.5-turbo", stream = True):
        
        system_content = system_content if system_content != None else self._determine_system_content(prompt)
            
        self.messages.append({"role": "system", "content": system_content}) 
        
        self.messages.append({"role": "user", "content": prompt})
        
        if stream:
            completion = self.client.chat.completions.create(
                model = model,
                messages = self.messages,
                stream = True
            )
            for chunk in completion:
                if chunk.choices[0].delta.content is not None:
                    print(chunk.choices[0].delta.content, end="")
        else: 
            completion = client.chat.completions.create(
                model = model,
                messages = self.messages
            )
            print(completion.choices[0].message.content)
            
    def _determine_system_content(self, content):
        if "summarize" in content:
            return self._summarizer()
        elif "translate" in content:
            return self._translator()
        elif "correct" in content:
            return self._english_grammar_master()
        elif "who is" in content:
            return self._who_is()
        elif "how to" in content:
            return self._how_to()
        else:
            return "You are a Wafa's personal AI Assistant that helps him with his daily tasks."
            
    def _summarizer(self):
        return """
                        You are a summarizer AI that summarizes the given content in two sentences.
                        The output format should be in a list
                    """.strip()

    def _translator(self):
        return """
                        You are a translator AI that translates the given content to user's requested language.
                        Don't make assumptions about what language the user wants to translate to.
                        Ask for clarification if a user doesn't specify a language.
                    """.strip()
        
    def _english_grammar_master(self):
        return """You are an English grammar master that corrects the given content.
                    """.strip()

    def _who_is(self):
        return """if the user asks about a person, return in this JSON format:
                            {
                                "name": "John Doe", # must be a full name
                                "skills": ["Mobile Development", "Web Development", "Machine Learning"],
                                "experience": [ # The experience must be clear information, if there is some missing information use empty string
                                    {
                                        "company": "Coolva Technology",
                                        "role": "Mobile Developer",
                                        "duration": 1 # must be return in year
                                    },
                                    {
                                        "company": "Gojek",
                                        "role": "Software Engineer",
                                        "duration": 1 # must be return in year
                                    }
                                ]
                            }
                            
                            if the user ask about How To Do something, 
                         """.strip()
        
    def _how_to(self):
        return """please return in this JSON List format:
                            [
                                {
                                    "step" : "step name",
                                    "description": "step description",
                                    "tools" : ["tool 1", "tool 2"], # optional, if there is a tool that needed
                                    "difficulty": "" # can be easy, medium, hard, or empty string
                                }
                            ]
                    """.strip()
        
    def display_chat_history(self):
        for message in self.messages:
            print(message["role"], ":", message["content"])

In [69]:
my_assistant = MyAssistant(openai_client=client, extra_knowledges=extra_knowledges)

Hello, Miriko here. Anything i could help today?


In [65]:
my_assistant.chat("do you know who am i?")

Of course! You are Muhammad Wafa Al Ausath, also known as Wafa. You are a student majoring in Informatics Engineering at Lampung University. You have a strong interest in the field of information technology, especially mobile application development. You have experience working as a mobile application developer and have interned at several startup companies. You also have a GitHub profile (https://github.com/mikirinkode), a LinkedIn profile (www.linkedin.com/in/muhammadwafaalausath), and a personal website (https://mikirinkode.com). Is there anything specific you would like assistance with?

In [70]:
my_assistant.chat("who is wafa?")

{
    "name": "Wafa",
    "skills": ["Mobile Application Development", "Information Technology"],
    "experience": [
        {
            "company": "Coolva Technology",
            "role": "Mobile Application Developer",
            "duration": 1
        },
        {
            "company": "Freelance",
            "role": "Mobile Developer",
            "duration": ""
        },
        {
            "company": "Startup Companies",
            "role": "Mobile Developer Intern",
            "duration": ""
        }
    ]
}

In [72]:
my_assistant.chat("how to make a cup of tea?")

[
    {
        "step": "Boil water",
        "description": "Fill a kettle with water and bring it to a boil.",
        "tools": ["Kettle", "Water"],
        "difficulty": "easy"
    },
    {
        "step": "Prepare teabag",
        "description": "Place a teabag in a cup or mug.",
        "tools": ["Teabag", "Cup or mug"],
        "difficulty": "easy"
    },
    {
        "step": "Pour hot water",
        "description": "Pour the hot water into the cup or mug with the teabag.",
        "tools": ["Kettle", "Cup or mug"],
        "difficulty": "easy"
    },
    {
        "step": "Steep",
        "description": "Let the teabag steep in the hot water for a few minutes to allow the flavors to infuse.",
        "tools": ["Teabag", "Cup or mug"],
        "difficulty": "easy"
    },
    {
        "step": "Add desired additions",
        "description": "If desired, add milk, honey, or any other additions to taste.",
        "tools": ["Milk", "Honey", "Other additions"],
        "difficulty":

In [73]:
my_assistant.chat("i want 'hello, how are you?' be translated in another language")

Sure, I can help you with that. Please specify the language you would like to translate "hello, how are you?" to.

In [74]:
my_assistant.chat("in japanese please")

"Hello, how are you?" in Japanese can be translated as "こんにちは、元気ですか？" (Konnichiwa, genki desu ka?)

In [75]:
my_assistant.display_chat_history()

system : You are a Wafa's personal AI Assistant that helps him with his daily tasks. 
             Here some extra knowledges for you [ExternalKnowledge(title='Wafa', content='Muhammad Wafa Al Ausath, or commonly known as Wafa, is a student majoring in Informatics Engineering at Lampung University.\n    Wafa has a strong interest in the field of information technology, especially mobile application development.\n    He has experience as a mobile application developer at Coolva Technology and has worked as a freelance mobile developer.\n    He has also served as a mobile developer intern at several startup companies.\n    \n    github profile = https://github.com/mikirinkode\n    linkedin profile = www.linkedin.com/in/muhammadwafaalausath\n    personal website = https://mikirinkode.com')]
system : if the user asks about a person, return in this JSON format:
                            {
                                "name": "John Doe", # must be a full name
                           